In [1]:
import tensorflow as tf
import numpy as np
na = np.newaxis

import matplotlib.pyplot as plt

# 1. Inputs:
Domain $X = \mathbb{R}^R$ ($R$-Dimensional)
$T \subset X$


### Fixed:

#### General:
- R
- Tlims: (Rx2)
- Data D: (NxR) (all points in T)

#### Inducing points:
    -> fix nbr M 
    -> Z: (MxR) location
    -> u: (M) (each sample of function values at the inducing points is M dimensional) 

#### Hyperparameters ($\Theta$)
    -> fixed at first, might become be optimized later as well
$\Theta = (\gamma, \alpha_1,...,  \alpha_R, \overline{u})$


### Parameters:
variational dist at inducing points u: q(u) = N (u;m,S)

m: (M)

S: (MxM)

In [ ]:
# pseudo stuff, no working code!!!

# constants:
# - R, T, M
# - D (NxR)
# - Z (MxR)
# - Theta values (gamma, alphas, ustrich)

# placeholders:
# - m (M)
# - S (MxM)


# kernel stuff:

def kernel_function(X, Y):
    return K_XY

# kernels to compute: K_zz, K_zd, trace(K_dd)

def lower_bound(D, m, S, Theta, T):
    K_zz = ...
    K_zz_inv = ...
    
    return - region_integral(m, S, K_zz_inv, Z, Theta, T) + datapoint_expectations(D, m, S, K_zz_inv, Theta) - kl_term(m,S,K_zz, K_zz_inv, Theta) 

def kl_term(m, S, K_zz, K_zz_inv, Theta):
    return scalar_value_node

def datapoint_expectations(D, m, S, K_zz_inv, Theta):
    
    k_zd = ...
    k_dd = ...
    
    musqare_N = ...
    sigsquare_N = ...
    
    C = 0.57721... #Euler-Masceroni constant
    
    lookup values = ... # problem: how to implement lookup table
    
    return scalar_value_node

def region_integral(m, S, K_zz_inv, Z, Theta, T):
    Psi = ... (MxM) 
    return scalar_value_node

# Tensorflow Test

In [2]:
# inducing point locations
Z = np.linspace(0, 10, 11)[:,na]

In [3]:
# https://github.com/GPflow/GPflow/issues/439
def tf_tril_indices(N, k=0):
    M1 = tf.tile(tf.expand_dims(tf.range(N), axis=0), [N,1])
    M2 = tf.tile(tf.expand_dims(tf.range(N), axis=1), [1,N])
    mask = (M1-M2) >= -k
    ix1 = tf.boolean_mask(M2, tf.transpose(mask))
    ix2 = tf.boolean_mask(M1, tf.transpose(mask))
    return ix1, ix2

In [20]:
def ard_kernel(X1, X2, gamma=1., alphas=None):
    # X1:  (n1 x d)
    # X2:  (n2 x d)
    # out: (n1 x n2
    with tf.name_scope('ard_kernel'):
        if alphas is None:
            alphas = tf.ones([tf.shape(X1)[1]])
        return gamma * tf.reduce_prod(tf.exp(- (tf.expand_dims(X1, 1) - tf.expand_dims(X2, 0))**2 / (2 * tf.expand_dims(tf.expand_dims(alphas, 0), 0))), axis=2) 

def kl_term(m, S, K_zz, K_zz_inv, u_ovln):
    # mean_diff = (u_ovln * tf.ones([tf.shape(Z_ph)[0]]) - m)
    mean_diff = tf.expand_dims(u_ovln * tf.ones([tf.shape(Z_ph)[0]]) - m, 1)
    first  = tf.trace(tf.matmul(K_zz_inv, S))
    second = tf.log(tf.matrix_determinant(K_zz) / tf.matrix_determinant(S))
    third  = tf.to_float(tf.shape(m)[0])
    # fourth = tf.reduce_sum(tf.multiply(tf.reduce_sum(tf.multiply(mean_diff, tf.transpose(K_zz_inv)), axis=1) , mean_diff))
    
    fourth = tf.squeeze(tf.matmul(tf.matmul(tf.transpose(mean_diff), K_zz_inv), mean_diff))
    
    return 0.5 * (first  - second - third + fourth), first, second, third, fourth

In [22]:
tf.reset_default_graph()

Z_ph = tf.placeholder(tf.float32, [None, None], name='inducing_point_locations')
u_ph = tf.placeholder(tf.float32, [],           name='inducin_point_mean')
n_ph = tf.placeholder(tf.int32,   [],           name='number_samples')


num_inducing_points = 11# tf.shape(Z_ph)[0] TODO: was da tun?

# mean
m_init = tf.ones([num_inducing_points])
m = tf.Variable(m_init)

# vectorized version of covariance matrix S (ensure valid covariance matrix)
vech_size   = (num_inducing_points * (num_inducing_points+1)) / 2 
vech_indices= tf.transpose(tf_tril_indices(num_inducing_points))
L_vech_init = tf.ones([vech_size])

L_vech = tf.Variable(L_vech_init)

L_shape = tf.constant([num_inducing_points, num_inducing_points])

L_st = tf.SparseTensor(tf.to_int64(vech_indices), L_vech, tf.to_int64(L_shape))

# L = tf.sparse_to_dense(vech_indices,L_shape , L_vech)

L = tf.sparse_add(tf.zeros(L_shape), L_st)

S = tf.matmul(L, tf.transpose(L))

K_zz  = ard_kernel(Z_ph, Z_ph)
K_zz_inv = tf.matrix_inverse(K_zz)

with tf.name_scope('approximation'):
    kl_term_op, first, second, third, fourth = kl_term(m, S, K_zz, K_zz_inv, u_ph)
    tf.summary.scalar('kl_div', kl_term_op)
    
with tf.name_scope('optimization'):
    train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(kl_term_op)

with tf.name_scope('prior_sampling'):
    cov  = K_zz
    mean = u_ph * tf.ones([num_inducing_points])
    ind_point_dist = tf.contrib.distributions.MultivariateNormalFullCovariance(mean, cov)
    samples = ind_point_dist.sample(n_ph)
    
m_grad = tf.gradients(kl_term_op, [m])[0]  
L_vech_grad = tf.gradients(kl_term_op, [L_vech])[0]

    
merged = tf.summary.merge_all()

In [24]:
max_iterations = 10000

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('logs', sess.graph)
    
    for i in range(max_iterations):
        _, summary, kl, m_val, S_val, f,s,t,fo = sess.run([train_step, merged, kl_term_op, m, S, first, second, third, fourth], feed_dict={Z_ph:Z, u_ph:0.})
        writer.add_summary(summary, i)
        #print('------------------------')
        if i % 100 == 0:
            print('..........')
            print(kl)
            print(np.max(np.absolute(S_val)))
            print(np.all(np.linalg.eigvals(S_val) > 0))
            print(f,s,t,fo)
        #print(m_val)
        #print(Lv_val)
        #print(dm_val)
        #print(dLv_val)
        
        #äif kl < 0.1:
        #    print(S_val)
        #    print(Cov_val)

..........
21.3753
11.0
True
42.3256 -6.49135 11.0 4.93374
..........
19.8214
10.7762
True
40.0099 -5.76013 11.0 4.87269
..........
18.4589
10.56
True
38.0634 -5.03906 11.0 4.8153
..........
17.2444
10.3509
True
36.4022 -4.32583 11.0 4.76072
..........
16.1455
10.1484
True
34.9645 -3.6182 11.0 4.70835
..........
15.1376
9.95203
True
33.7035 -2.91397 11.0 4.65779
..........
14.202
9.76145
True
32.5842 -2.21099 11.0 4.60873
..........
13.3239
9.57821
True
31.5798 -1.50714 11.0 4.56097
..........
12.4921
9.50773
True
30.6697 -0.800249 11.0 4.51435
..........
11.6975
9.43542
True
29.8381 -0.0881347 11.0 4.46873
..........
10.9326
9.36149
True
29.0726 0.631524 11.0 4.42404
..........
10.1913
9.28611
True
28.3634 1.3611 11.0 4.38019
..........
9.46843
9.20946
True
27.703 2.10323 11.0 4.33713
..........
8.75955
9.13169
True
27.085 2.8607 11.0 4.2948
..........
8.0606
9.05292
True
26.5047 3.63665 11.0 4.25316
..........
7.36784
8.97328
True
25.9581 4.43462 11.0 4.21218
..........
6.67761
8.892

InvalidArgumentError: Input is not invertible.
	 [[Node: optimization/gradients/approximation/MatrixDeterminant_1_grad/MatrixInverse = MatrixInverse[T=DT_FLOAT, adjoint=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](MatMul)]]

Caused by op 'optimization/gradients/approximation/MatrixDeterminant_1_grad/MatrixInverse', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
 

### example sampling:

In [ ]:
# sampling
num_samples = 10

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('logs', sess.graph)
    sample_res, mean_res, cov_res = sess.run([samples, mean, cov], feed_dict={Z_ph:Z, u_ph:0., n_ph:num_samples})

In [ ]:
for i in range(num_samples):
    plt.plot(Z, sample_res[i])
plt.show()